# DALL-E - Generating Images from Text

In [2]:
COLLAB = True

In [ ]:
if COLAB:
  # --- Remember to change the runtime to use GPU for better performance
  !nvidia-smi
  !pip install -Uqq ftfy regex tqdm 
  !rm -rf jupyter-notebooks && git clone https://github.com/alpha2phi/jupyter-notebooks.git 
  !rm -rf CLIP && cp -R jupyter-notebooks/nbs/CLIP .
  !rm -rf test_data && cp -R jupyter-notebooks/nbs/test_data .
else:
  !pip install -Uqq ftfy regex tqdm torch torchvision